In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import urllib2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

First, define X

In [29]:
target_data = "./features.csv"
features = pd.read_csv(target_data, parse_dates = True)
features = features.drop(['Unnamed: 0'],axis=1)
#features = features.drop(['Operating Margin', 'P/B', 'ROA', 'EV', 'Shares Outstanding'],axis=1)
#features = features.drop(['Revenue'],axis=1)
print features.info()
print "--------------------------------------------------------"

X = []
for index, rows in features.iterrows():
    X.append([])
    X[index].append(1)
    for a in range(1, len(rows)):
        X[index].append(rows[a])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 26 columns):
Quote                       1459 non-null object
Beta                        1459 non-null float64
Current Ratio               1459 non-null float64
Diluted EPS                 1459 non-null float64
EBITDA                      1459 non-null float64
EV                          1459 non-null float64
Operating Margin            1459 non-null float64
P/B                         1459 non-null float64
P/S                         1459 non-null float64
Profit Margin               1459 non-null float64
Qualterly Revenue Growth    1459 non-null float64
ROA                         1459 non-null float64
ROE                         1459 non-null float64
Week Change                 1459 non-null float64
isBasicIndustries           1459 non-null int64
isCapitalGoods              1459 non-null int64
isConsumerDurables          1459 non-null int64
isConsumerNonDurables       1459 non-null int64
is

In [30]:
features.head()

Quote  Beta  Current Ratio  Diluted EPS        EBITDA            EV  \
0  AABA  1.98          17.07        29.96  6.650000e+06  5.905000e+10   
1   AAL  1.27           0.61         3.90  6.790000e+09  4.170000e+10   
2  AAME  0.77           1.21         0.20  8.520000e+06  7.366000e+07   
3  AAOI  1.22           3.25         3.67  1.073300e+08  5.553100e+08   
4  AAON  0.67           3.07         1.03  8.988000e+07  1.820000e+09   

   Operating Margin   P/B      P/S  Profit Margin        ...         \
0            0.1221  0.95  1159.80       453.8760        ...          
1            0.1130  5.78     0.54         0.0455        ...          
2            0.0391  0.56     0.35         0.0250        ...          
3            0.2274  1.65     1.44         0.1934        ...          
4            0.1848  7.99     4.68         0.1345        ...          

   isConsumerDurables  isConsumerNonDurables  isConsumerServices  isEnergy  \
0                   0                      0                   0         0   
1                   0                      0                   0         0   
2                   0                      0                   0         0   
3                   0                      0                   0         0   
4                   0                      0                   0         0   

   isFinance  isHealthCase  isMiscellaneous  isPublicUtilities  isTechnology  \
0          0             0                0                  0             1   
1          0             0                0                  0             0   
2          1             0                0                  0             0   
3          0             0                0                  0             1   
4          0             0                0                  0             0   

   isTransportation  
0                 0  
1                 1  
2                 0  
3                 0  
4                 0  

[5 rows x 26 columns]

Define Y and filter out extra variables

In [31]:
target_data = "./stock01.csv"
stock01 = pd.read_csv(target_data, parse_dates = True)
stock01 = stock01.drop(['Unnamed: 0'],axis=1)
print stock01.info()
print "------------------------------------------------------"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 6 columns):
Stock              2660 non-null object
Bollinger Band     2660 non-null int64
Keltner Channel    2660 non-null int64
Moving Envelope    2660 non-null int64
Parabolic SAR      2660 non-null int64
Price Channel      2660 non-null int64
dtypes: int64(5), object(1)
memory usage: 124.8+ KB
None
------------------------------------------------------


In [32]:
overlays = pd.DataFrame({'Stock':features.Quote})
overlays.head()

Stock
0  AABA
1   AAL
2  AAME
3  AAOI
4  AAON

In [33]:
overlays = pd.merge(stock01, overlays, on='Stock', how='inner')

In [34]:
overlays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 6 columns):
Stock              1459 non-null object
Bollinger Band     1459 non-null int64
Keltner Channel    1459 non-null int64
Moving Envelope    1459 non-null int64
Parabolic SAR      1459 non-null int64
Price Channel      1459 non-null int64
dtypes: int64(5), object(1)
memory usage: 79.8+ KB


In [35]:
y_pc = overlays['Price Channel'].tolist()
y_mae = overlays['Moving Envelope'].tolist()
y_bb = overlays['Parabolic SAR'].tolist()
y_kc = overlays['Keltner Channel'].tolist()
y_psar = overlays['Bollinger Band'].tolist()

Build the Logistic regression model

In [36]:
# The probability of price channels
X_train, X_test, y_train, y_test = train_test_split(X, y_pc, test_size=0.3, random_state=1)

lr_pc = LogisticRegression(solver='liblinear')
result_pc = lr_pc.fit(X_train, y_train)

print "Price Channels"
print "Log Regression train: {}".format(np.mean(lr_pc.predict(X_train) == y_train))
print "Log Regression test: {}".format(np.mean(lr_pc.predict(X_test) == y_test))

Price Channels
Log Regression train: 0.611165523996
Log Regression test: 0.60502283105


In [37]:
# The probability of Moving Average Envelope
X_train, X_test, y_train, y_test = train_test_split(X, y_mae, test_size=0.3, random_state=1)

lr_mae = LogisticRegression(solver='liblinear')
result_mae= lr_mae.fit(X_train, y_train)

print "Moving Average Envelope"
print "Log Regression train: {}".format(np.mean(lr_mae.predict(X_train) == y_train))
print "Log Regression test: {}".format(np.mean(lr_mae.predict(X_test) == y_test))

Moving Average Envelope
Log Regression train: 0.715964740451
Log Regression test: 0.75799086758


In [38]:
# The probability of Bolling Bands
X_train, X_test, y_train, y_test = train_test_split(X, y_bb, test_size=0.3, random_state=1)

lr_bb = LogisticRegression(solver='liblinear')
result_bb = lr_bb.fit(X_train, y_train)

print "Bollinger Bands"
print "Log Regression train: {}".format(np.mean(lr_bb.predict(X_train) == y_train))
print "Log Regression test: {}".format(np.mean(lr_bb.predict(X_test) == y_test))

Bollinger Bands
Log Regression train: 0.86777668952
Log Regression test: 0.842465753425


In [39]:
# The probability of Keltner Channels
X_train, X_test, y_train, y_test = train_test_split(X, y_kc, test_size=0.3, random_state=1)

lr_kc = LogisticRegression(solver='liblinear')
result_kc = lr_kc.fit(X_train, y_train)

print "Keltner Channels"
print "Log Regression train: {}".format(np.mean(lr_kc.predict(X_train) == y_train))
print "Log Regression test: {}".format(np.mean(lr_kc.predict(X_test) == y_test))

Keltner Channels
Log Regression train: 0.869735553379
Log Regression test: 0.840182648402


In [40]:
# The probability of Parabolic SAR
X_train, X_test, y_train, y_test = train_test_split(X, y_psar, test_size=0.3, random_state=1)

lr_psar = LogisticRegression(solver='liblinear')
result_psar = lr_psar.fit(X_train, y_train)

print "Parabolic SAR"
print "Log Regression train: {}".format(np.mean(lr_psar.predict(X_train) == y_train))
print "Log Regression test: {}".format(np.mean(lr_psar.predict(X_test) == y_test))

Parabolic SAR
Log Regression train: 0.861900097943
Log Regression test: 0.86301369863
